# Import

In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [41]:
raw_mail_data = pd.read_csv('../data/mail_dataset.csv')

# EDA and Preprocessing

In [42]:
raw_mail_data.head()

,id,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [34]:
raw_mail_data.shape

(5171, 4)

In [35]:
raw_mail_data.isnull().sum()

id           0
label        0
text         0
label_num    0
dtype: int64

In [36]:
raw_mail_data = raw_mail_data.drop(columns=['id'], inplace=True)

In [39]:
X = raw_mail_data['text']
y = raw_mail_data['label_num']

TypeError: 'NoneType' object is not subscriptable